In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np

# Load the pretrained SSD MobileNet model from TensorFlow Hub

In [2]:
detector = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Load COCO labels

In [3]:
labels_path = tf.keras.utils.get_file(
    'coco_labels.txt',
    'https://raw.githubusercontent.com/amikelive/coco-labels/master/coco-labels-paper.txt'
)
labels = [line.strip() for line in open(labels_path).readlines()]

# Start webcam feed

In [4]:
# %pip install opencv-python

# import cv2  # Ensure cv2 is imported after installation

cap = cv2.VideoCapture(0)  # Use camera index 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess the frame: resize, normalize, expand dims
    input_tensor = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)          # Convert BGR to RGB
    input_tensor = tf.convert_to_tensor(input_tensor, dtype=tf.uint8)
    input_tensor = tf.image.resize(input_tensor, (320, 320))
    input_tensor = tf.cast(input_tensor, tf.uint8)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run detection
    detections = detector(input_tensor)
    detections = {k: v.numpy() for k, v in detections.items()}

    # Draw detections with score > 0.5
    h, w, _ = frame.shape
    num_detections = int(detections['num_detections'][0])
    for i in range(num_detections):
        score = detections['detection_scores'][0][i]
        if score < 0.5:
            continue

        box = detections['detection_boxes'][0][i]
        class_id = int(detections['detection_classes'][0][i])
        label = labels[class_id]

        ymin, xmin, ymax, xmax = box
        left, top = int(xmin * w), int(ymin * h)
        right, bottom = int(xmax * w), int(ymax * h)

        # Draw rectangle and label
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} {score:.2f}", (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Show frame
    cv2.imshow('Real-Time Object Detection', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()